In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras
import numpy as np
import re
import os 
conn=psycopg2.connect(database="blackmetal",
                        host="localhost",
                        user="bm_admin",
                        password="18cba9cd-0776-4f09-9c0e-41d2937fab2b",
                        port=5432) 


In [2]:
file = "Sunioj-music-export.csv"
data = pd.read_csv(file).fillna("")
data.columns = [i.strip() for i in data.columns]

In [3]:
rated = data[data["Rating"].astype("int32") > 8].copy()
artists = []

for something,artist in zip(rated["First Name"].str.strip().tolist(),rated["Last Name"].str.strip().tolist()):
    artist = something + " " + artist if len(something) > 0 else artist
    artists.append(artist)

rated["Artist"] = artists
rated = rated[["Title","Release_Date","Artist"]]

In [4]:
filtered = rated[rated["Artist"].isin(["Ascension","Abigor","The Black","Corpus Christii","Imperium Dekadenz","Kataxu","Krohm","Azarath","Warmoon Lord","Deus Mortem","Deströyer 666","Mayhem","Walknut","Sielunvihollinen","Dissection","Necrophobic","Nokturnal Mortum"])]

In [5]:
def create_artists(filtered):
    cursor = conn.cursor()
    command = "insert into artists (name) values\n"
    for n,artist in enumerate(filtered.Artist.unique()):
        value = "('%s')" % artist
        terminator = ";" if n+1 == len(filtered.Artist.unique()) else ",\n"
        value += terminator
        command += value
    print(command)
    cursor.execute(command)
    conn.commit()   

In [6]:
def get_artists():
    cursor = conn.cursor()
    command = "select artist_id, name from artists;"
    cursor.execute(command)
    data = cursor.fetchall()
    conn.commit()
    return pd.DataFrame(data,columns=["artist_id","Artist"])

In [7]:
artists = get_artists()


In [176]:
create_artists(filtered)

insert into artists (name) values
('Ascension'),
('The Black'),
('Corpus Christii'),
('Imperium Dekadenz'),
('Kataxu'),
('Krohm'),
('Warmoon Lord'),
('Azarath'),
('Deströyer 666'),
('Mayhem'),
('Deus Mortem'),
('Walknut'),
('Sielunvihollinen'),
('Abigor'),
('Dissection'),
('Nokturnal Mortum'),
('Necrophobic');


In [8]:
with_ids = filtered.merge(artists, on='Artist', how='left')
with_ids["price"] = np.random.randint(8,high=20, size=(len(with_ids))) + np.random.rand(len(with_ids))
with_ids["stock"] = np.random.randint(0,high=10, size=(len(with_ids)))

In [9]:
photos = os.listdir("photos")

In [15]:
photo_arr = []

for artist,title in with_ids[["Artist","Title"]].values:
    photo_str = (artist.lower() + " " + title.lower()).replace(" ","-")+"-Cover-Art.webp"
    new_str = re.sub("[\'\(\)]","",photo_str).replace("ö","o").replace("ä","a")
    if new_str in photos:
        photo_arr.append(new_str)
    else:
        print(new_str)

In [16]:
with_ids["photo"] = photo_arr

In [18]:
def insert_albums(with_ids):
    #[["Title","Release_Date","stock","price","photo","artist_id"]].values
    cursor = conn.cursor()
    command = "insert into albums (title,release_year,stock,price,photo,artist_id) values\n"
    for n,row in enumerate(with_ids.to_dict(orient="records")):
        value = "('%s',%s,%s,%s,'%s',%s)" % (row["Title"].replace("'","''"),row["Release_Date"],row["stock"],round(row["price"],2),row["photo"],row["artist_id"])
        terminator = ";" if n+1 == len(with_ids) else ",\n"
        value += terminator
        command += value
    cursor.execute(command)
    conn.commit()   
    print(command)

In [19]:
insert_albums(with_ids)

insert into albums (title,release_year,stock,price,photo,artist_id) values
('The Dead of the World',2014,5,9.27,'ascension-the-dead-of-the-world-Cover-Art.webp',100),
('Alongside Death',2008,0,8.46,'the-black-alongside-death-Cover-Art.webp',101),
('Rising',2007,2,9.33,'corpus-christii-rising-Cover-Art.webp',102),
('Procella Vadens',2010,5,15.95,'imperium-dekadenz-procella-vadens-Cover-Art.webp',103),
('Hunger of Elements',2005,3,17.66,'kataxu-hunger-of-elements-Cover-Art.webp',104),
('The Haunting Presence',2007,3,12.69,'krohm-the-haunting-presence-Cover-Art.webp',105),
('Battlespells',2021,1,18.17,'warmoon-lord-battlespells-Cover-Art.webp',106),
('Blasphemers'' Maledictions',2011,3,16.75,'azarath-blasphemers-maledictions-Cover-Art.webp',107),
('Defiance',2009,6,15.97,'destroyer-666-defiance-Cover-Art.webp',108),
('De mysteriis dom Sathanas',1994,6,19.25,'mayhem-de-mysteriis-dom-sathanas-Cover-Art.webp',109),
('Emanations of the Black Light',2013,6,19.76,'deus-mortem-emanations-of-the-

In [20]:
songs = pd.read_csv("songs.csv",header=None)
songs.columns = ["raw","title"]

In [21]:
def get_albums():
    cursor = conn.cursor()
    command = "select album_id, title from albums;"
    cursor.execute(command)
    data = cursor.fetchall()
    return pd.DataFrame(data,columns=["album_id","title"])

In [22]:
data = get_albums()

In [23]:
def get_songs_frame(songs):
    song_pattern = r"^[0-9]{2}|^[0-9]{1}"
    dur_pattern = r"\d:\d{2}|\d{2}:\d{2}"
    song_nros = []
    song_names = []
    song_durs = []
    for raw in songs.raw:
        song_nro = int(re.search(song_pattern, raw).group())
        rest = " ".join(re.split(song_pattern, raw))
        dur_match = re.search(dur_pattern,rest)
        song_dur = np.nan
        if dur_match != None:
            min_sec = dur_match.group().split(":")
            song_dur = int(min_sec[0]) * 60 +  int(min_sec[1])
        song_name = "".join(re.split(dur_pattern,rest)).strip()
        song_nros.append(song_nro)
        song_names.append(song_name)
        song_durs.append(song_dur)
        #print(song_nro + " " + str(song_dur) + " " + song_name)
    return pd.DataFrame(data={"song":song_names,"duration":song_durs,"track":song_nros})


In [24]:
re_songs = get_songs_frame(songs)
combined = pd.concat([songs,re_songs],axis=1).drop(columns=["raw"])
final = combined.merge(data, on='title', how='left').drop(columns=["title"])

In [25]:
def insert_songs(final):
    cursor = conn.cursor()
    command = "insert into songs (track,album_id,song,duration) values\n"
    for n,row in enumerate(final.to_dict(orient="records")):
        escaped_song = row["song"].replace("'","''")
        nulled = "NULL" if np.isnan(row["duration"]) else int(row["duration"])
        value = "(%s,%s,'%s',%s)" % (row["track"],row["album_id"],escaped_song,nulled)
        terminator = ";" if n+1 == len(final) else ",\n"
        value += terminator
        command += value
    cursor.execute(command)
    conn.commit()   
    print(command)
    

In [26]:
insert_songs(final)

insert into songs (track,album_id,song,duration) values
(1,1022,'The Silence of Abel',NULL),
(2,1022,'Death''s Golden Temple',NULL),
(3,1022,'Black Ember',NULL),
(4,1022,'Unlocking Tiamat',NULL),
(5,1022,'Deathless Light',NULL),
(6,1022,'The Dark Tomb Shines',NULL),
(7,1022,'Mortui Mundi',NULL),
(1,1023,'On the Descent to Hell',NULL),
(2,1023,'Death''s Crown',NULL),
(3,1023,'A Contract Written in Ashes',NULL),
(4,1023,'Dead Seed',NULL),
(5,1023,'Fleshless',NULL),
(6,1023,'Death Throes',NULL),
(7,1023,'The Wrath From Beneath',NULL),
(8,1023,'Alongside Death',NULL),
(1,1024,'Intro',104),
(2,1024,'Stabbed',272),
(3,1024,'Blank Code',245),
(4,1024,'Black Gleam Eye',264),
(5,1024,'The Wanderer',370),
(6,1024,'Torrents of Sorrow',349),
(7,1024,'Void Revelation',305),
(8,1024,'Evasive Contempt',238),
(9,1024,'Heavenless Bliss',365),
(10,1024,'Untouchable Euphoria',201),
(11,1024,'Bleak Existence',259),
(12,1024,'Revealed Wounds',391),
(13,1024,'Outro',101),
(1,1025,'Die Hoffnung stirbt...',12